In [1]:
import pandas as pd
import numpy as np

## Internal Data

In [2]:
np.random.seed(42)
dates = pd.date_range('2018-01-01', '2023-12-01', freq='M')
n = len(dates)

# Simulate cumulative balances
balances = {
    'VR_balance': 1000 + np.cumsum(np.random.normal(20, 5, n)),
    'Fixed_balance': 1500 + np.cumsum(np.random.normal(15, 7, n)),
    'Redeemable_balance': 800 + np.cumsum(np.random.normal(25, 10, n))
}

# Simulate monthly pricing levels (e.g. interest or price index)
prices = {
    'VR_price': np.random.uniform(0.01, 0.05, n),
    'Fixed_price': np.random.uniform(0.02, 0.06, n),
    'Redeemable_price': np.random.uniform(0.015, 0.055, n)
}

internal_df = pd.DataFrame({**balances, **prices}, index=dates)
internal_df.index.name = 'Date'

# View the first 5 rows
print(internal_df.head())

            Fixed_balance  Fixed_price  Redeemable_balance  Redeemable_price  \
Date                                                                           
2018-01-31    1525.766256     0.054260          808.925168          0.018648   
2018-02-28    1540.515474     0.046348          835.771506          0.034777   
2018-03-31    1566.467979     0.026517          863.370334          0.017302   
2018-04-30    1563.129764     0.022823          896.188563          0.036981   
2018-05-31    1583.883081     0.045697          908.819056          0.032661   

             VR_balance  VR_price  
Date                               
2018-01-31  1022.483571  0.045600  
2018-02-28  1041.792249  0.023520  
2018-03-31  1065.030692  0.025023  
2018-04-30  1092.645841  0.013759  
2018-05-31  1111.475074  0.033131  


In [3]:
internal_df.to_excel('fake_internal.xlsx')

## Model MEV

In [4]:
def generate_raw_mev_tables(
    start: str = '2000Q1',
    end: str = '2030Q4',
    data_defs: dict = None,
    scenario_factors: dict = None
) -> dict:
    """
    Generate raw MEV tables (with multi-line headers) for a base model and multiple scenarios.

    Parameters
    ----------
    start : str
        Start quarter in 'YYYYQn' format, e.g. '2000Q1'.
    end : str
        End quarter in 'YYYYQn' format, e.g. '2030Q4'.
    data_defs : dict
        Mapping of code -> (base level generator), e.g. {
            'NGDP': lambda n: 10000+np.linspace(0,500,n)+np.random.normal(0,50,n),
            'PSR':  lambda n: np.random.uniform(3,8,n),
            ...
        }
    scenario_factors : dict
        Mapping of scenario_name -> dict of code->multiplier.
        Example:
          {
            'base': {'NGDP':1.0, 'PSR':1.0, ...},
            'adv':  {'NGDP':0.95, 'PSR':0.95, ...},
            'sev':  {'NGDP':0.9 , 'PSR':0.9 , ...},
          }

    Returns
    -------
    raw_tables : dict
        Mapping of sheet_name -> raw DataFrame ready for Excel export.
    """
    # 1) Prepare quarterly index
    periods = pd.period_range(start, end, freq='Q')
    n = len(periods)

    # 2) Build base data
    if data_defs is None:
        data_defs = {
            'NGDP':   lambda n: 10000 + np.linspace(0, 500, n) + np.random.normal(0, 50, n),
            'PSR':    lambda n: np.random.uniform(3, 8, n),
            'PDI':    lambda n: 8000  + np.linspace(0, 300, n) + np.random.normal(0, 30, n),
            'UNRATE': lambda n: np.random.uniform(3, 10, n),
            'CPI':    lambda n: 100   + np.linspace(0, 5, n)   + np.random.normal(0, 0.3, n),
        }
    df_base = pd.DataFrame(
        {code: gen(n) for code, gen in data_defs.items()},
        index=periods.to_timestamp(how='end')
    )
    df_base.index.name = 'Date'

    # 3) Default scenario factors if none provided
    if scenario_factors is None:
        scenario_factors = {
            'base': {c:1.0 for c in data_defs},
            'adv' : {c:(0.95 if c in ['NGDP','PSR'] else 1.1) for c in data_defs},
            'sev' : {c:(0.90 if c in ['NGDP','PSR'] else 1.2) for c in data_defs},
        }

    # 4) Prepare header rows
    codes = list(data_defs.keys())
    # descriptive names derived from codes
    desc_names = {
        'NGDP': 'Nominal GDP (Billions)',
        'PSR':  'Personal Savings Rate (%)',
        'PDI':  'Personal Disposable Income (Billions)',
        'UNRATE': 'Unemployment Rate (%)',
        'CPI': 'Consumer Price Index'
    }
    # Use mapping or code itself
    names = [desc_names.get(c, c) for c in codes]

    top_row = ['Metadata'] + [''] * len(codes)
    mid_row = ['Metadata2'] * (1 + len(codes))
    desc_row = ['Date', ''] + [f'Canada\n{names[i]}' for i in range(len(codes))]
    code_row = ['', ''] + codes

    # 5) Build raw tables for each scenario
    raw_tables = {}
    for sce, factors in scenario_factors.items():
        df = df_base.copy()
        for c, f in factors.items():
            df[c] = (df[c] * f).round(2)
        # Data rows labeled 'YYYY:Qn'
        data_rows = [
            [f"{ts.year}:{ts.quarter}", ''] + list(vals)
            for ts, vals in df.iterrows()
        ]
        raw = pd.DataFrame([top_row, mid_row, desc_row, code_row, code_row, code_row] + data_rows)
        raw_tables[sce] = raw

    return raw_tables

In [5]:
raw_tables = generate_raw_mev_tables()
with pd.ExcelWriter('fake_scens.xlsx') as writer:
    for sheet, df in raw_tables.items():
        df.to_excel(writer, sheet_name=sheet, index=False, header=False)